In [1]:
# Import necessary libraries
import cv2
import numpy as np

In [2]:
# Initialize video capture
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Get the frame rate and total frame count to calculate video duration
fps = video.get(cv2.CAP_PROP_FPS)
video_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
video_duration_seconds = video_frames / fps
video_duration_minutes = video_duration_seconds / 60

# Set the frame skip rate (For debugging purposes)
normal_frame_skip_rate = 1
fast_forward_skip_rate = 10
frame_skip_rate = normal_frame_skip_rate

initial_frame = None # Initialize the initial frame for comparison
car_count = 0 # Track number of cars that pass through the red line
line_position = 670 # Vertical line position to track cars passing through
car_positions = {} # Dictionary to store car positions
next_car_id = 0  # To assign unique IDs to each car
centroid_buffer = []  # Buffer to track centroids for duplicate counting

# Create an infinite loop to keep reading frames
while True:
    check, frame = video.read()
    if not check:
        break

    # If frame_skip_rate more than 1, skip frames based on frame_skip_rate
    if frame_skip_rate > 1:
        for _ in range(frame_skip_rate - 1):
            video.grab()  # Skip frames

    # Convert the frame to grayscale and use gaussian blur to reduce noise
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)

    # Save the first frame as the initial frame
    if initial_frame is None:
        initial_frame = blur_frame

    # Compute the difference between the initial frame and the current frame
    delta_frame = cv2.absdiff(initial_frame, blur_frame)

    # Convert the difference (delta_frame) to a binary image.
    # If the pixel value is greater than 30, it will white, else it will be black.
    threshold, threshold_frame = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)

    # Apply morphological closing to close small gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
    closed_frame = cv2.morphologyEx(threshold_frame, cv2.MORPH_CLOSE, kernel)

    # Find contours in the thresholded frame
    contours, threshold = cv2.findContours(closed_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through each detected contour
    for contour in contours:
        if cv2.contourArea(contour) < 1500:  # Ignore small contours
            continue

        # Get the bounding box around the contour, then calculate the centroid
        (x, y, w, h) = cv2.boundingRect(contour)
        centroid = (x + w // 2, y + h // 2)

        # Loop through each car's position and calculate the Euclidean distance between the
        # new centroid and the prev_centroid.
        car_matched = False
        for car_id, prev_centroid in car_positions.items():
            distance = np.linalg.norm(np.array(centroid) - np.array(prev_centroid))

            # If the distance is less than 20, assume its the same car and update its position
            if distance < 20: 
                car_positions[car_id] = centroid
                car_matched = True

                # Check if the car crossed the line from right to left
                if prev_centroid[0] > line_position and centroid[0] <= line_position:

                    # If the car is not in the buffer, increment car_count and add the car_id to the buffer.
                    if car_id not in centroid_buffer:
                        car_count += 1
                        centroid_buffer.append(car_id)
                break

        # If car is not found in car_positions, add a new car to car_positions
        if not car_matched:
            car_positions[next_car_id] = centroid
            next_car_id += 1

        # Draw a green rectangle around the detected car
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Draw the red vertical line
    cv2.line(frame, (line_position, 0), (line_position, int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))), (0, 0, 255), 3)

    # Display the frame with the bounding boxes and vertical line
    cv2.putText(frame, f"Car Count: {car_count}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow("Detected Cars", frame)

    # Check for key press events
    key = cv2.waitKey(30) & 0xFF  # Capture the pressed key
    if key == 27:  # Exit when "Esc" key is pressed
        break
    elif key == ord('d'):
        frame_skip_rate = fast_forward_skip_rate
    elif key == ord('a'):
        frame_skip_rate = normal_frame_skip_rate

# Release resources and close the window
video.release()
cv2.destroyAllWindows()

# Calculate cars per minute
cars_per_minute = car_count / video_duration_minutes

# Print total number of cars detected
print("Total number of cars", car_count)
print("Cars per minute:", round(cars_per_minute, 2))

Total number of cars 0
Cars per minute: 0.0
